In [1]:
import stanza.runtime
stanza.runtime.setup() # setup logging, etc.

import jax
import jax.flatten_util
import matplotlib.pyplot as plt
import jax.numpy as jnp

from stanza.diffusion import DDPMSchedule
from stanza.datasets import image_class_datasets

DATASET = "mnist"

dataset = image_class_datasets.create(DATASET)

schedule = DDPMSchedule.make_squaredcos_cap_v2(100)

In [ ]:
from stanza.nn.unet import DiffusionUNet
from stanza import train as tu

model = DiffusionUNet(embed_dim=32)
init_params = model.init(
    jax.random.PRNGKey(42),
    dataset.splits["train"][0], timestep=0)

def loss_fn(params, rng_key, sample):
    denoiser = lambda _, x, t: model.apply(params, x, timestep=t)
    loss = schedule.loss(rng_key, denoiser, sample)
    return tu.LossOutput(
        loss=loss,
        stats={"loss": loss}
    )

trained_params = 